# Convert forecasts
This notebook is used to convert quantile forecasts to a parametric forecast that can be used as the input for the optimization process.

## Data Read-In

In [1]:
import pandas as pd
import numpy as np
import glob  as glob
import os
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import plotly.graph_objects as go
import scipy.optimize
import inspect
import os
from datetime import datetime

In [ ]:
path_forecast = '../data/quantile_forecasts/infinite_horizon_forecast/patch_tst_2025-03-04_12-40-56_prosumption_hour_1.csv'
path_gt = '../data/ground_truth/residential4_prosumption.csv'

def read_in_data(path):
    forecast = pd.read_csv(path, index_col=0, parse_dates=True)
    forecast.index = pd.to_datetime(forecast.index)
    return forecast

forecast = read_in_data(path_forecast)
forecast.columns = forecast.columns.str.replace('0_', '') # rename columns 0_0.01- 0_0.99 to 0.01 - 0.99
forecast.columns = forecast.columns.astype(float)

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')

gt = read_in_data(path_gt)

## Visualization

In [3]:
def plot_specific_day(fc_df, gt_df, day, title=None):
    plt.figure(figsize=(16,8))
    plt.plot(fc_df.loc[day], label='Forecast', alpha=0.1)
    plt.plot(gt_df.loc[day], label='Ground Truth', alpha=1)
    plt.ylabel('Power in kW')
    plt.xlabel('Time')
    if title is not None:
        plt.title(title)
    plt.show()

# create an viszualisation of every hour given a forecast containing the quantiles 0.01 to 0.99
def plot_forecast_hour(fc_df, gt_df, day_and_hour, title=None):
    plt.figure(figsize=(16,8))
    plt.plot(fc_df.loc[day_and_hour].values, fc_df.columns, label='Forecast', alpha=1)
    # plot the ground truth as horizontal line
    gt_value = gt_df.loc[day_and_hour].values[0]
    plt.axhline(y=0, color='k', linestyle='-')
    plt.vlines(x=gt_value, ymin=0, ymax=0.9, color='r', linestyle='-', label='Ground Truth')
    plt.ylabel('Quantiles')
    plt.yticks(np.arange(0.01, 1.01, 0.02))
    plt.xlabel('Power in kW')
    if title is not None:
        plt.title(title)   
    plt.legend()
    plt.show()


date_to_visualize = '2017-05-18'
date_time_to_visualize = date_to_visualize + ' 10:00:00'
#plot_specific_day(forecast, gt, date_to_visualize, title=f'Quantile Forecast and Ground Truth for {date_to_visualize}')
#plot_forecast_hour(forecast, gt, date_time_to_visualize, title=f'Forecasted CDF and Ground Truth for {date_time_to_visualize}')

## Sort the Quantiles

In [4]:
# go through every row of the dataframe and ensure that the values in ascending order (should be the case already => see np.quantile() docs)
def sort_quantiles(fc_df):
    fc_sorted = []
    for index, row in fc_df.iterrows():
        # sort the values in ascending order
        sorted_values = row.sort_values()
        fc_sorted.append(sorted_values.values)
    return pd.DataFrame(fc_sorted, index=fc_df.index, columns=fc_df.columns)

forecast_sorted = sort_quantiles(forecast)

# plot the sorted forecast
#plot_specific_day(forecast_sorted, gt, date_to_visualize, title=f'Sorted Quantile Forecast and Ground Truth for {date_to_visualize}')
#plot_forecast_hour(forecast_sorted, gt, date_time_to_visualize, title=f'Sorted Forecasted CDF and Ground Truth for {date_time_to_visualize}')

## Smooth the quantiles

In [5]:
def smooth_quantiles(fc_df, window_size=2): # Paper: Window size of 2 was used
    fc_smoothed = []
    for index, row in fc_df.iterrows():
        smoothed_values = row.rolling(window=window_size, min_periods=1, center=True).mean()
        fc_smoothed.append(smoothed_values.values)
    return pd.DataFrame(fc_smoothed, index=fc_df.index, columns=fc_df.columns)

forecast_smoothed = smooth_quantiles(forecast_sorted)

#plot_specific_day(forecast_smoothed, gt, date_to_visualize, title=f'Smoothed Quantile Forecast and Ground Truth for {date_to_visualize}')
#plot_forecast_hour(forecast_smoothed, gt, date_time_to_visualize, title=f'Smoothed Forecasted CDF and Ground Truth for {date_time_to_visualize}')

In [6]:
forecast_smoothed

,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,...,0.90,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99
cest_timestamp,,,,,,,,,,,,,,,,,,,,,
2017-04-29 00:00:00,-0.198469,-0.113437,0.018515,0.098603,0.159060,0.203635,0.239368,0.258968,0.260973,0.276829,...,0.892303,0.915137,0.936326,0.966912,0.997218,1.033416,1.106704,1.173262,1.202981,1.257560
2017-04-29 01:00:00,-0.258353,-0.189924,-0.087790,-0.033846,0.026413,0.077102,0.125074,0.169498,0.182604,0.217864,...,0.810180,0.830299,0.868612,0.904821,0.937492,0.978166,1.025418,1.090705,1.164604,1.306835
2017-04-29 02:00:00,-0.401059,-0.283698,-0.105407,0.004814,0.072836,0.096145,0.119694,0.156508,0.185139,0.210426,...,0.852452,0.869443,0.879642,0.928014,0.993625,1.042486,1.095206,1.160425,1.271612,1.439424
2017-04-29 03:00:00,-0.420835,-0.307391,-0.116667,0.024480,0.122909,0.164751,0.180752,0.193506,0.220038,0.242858,...,0.852153,0.886729,0.909304,0.934423,0.982687,1.048472,1.114674,1.234714,1.325369,1.450811
2017-04-29 04:00:00,-0.503812,-0.368100,-0.125743,0.006878,0.107101,0.190914,0.231001,0.268104,0.277914,0.289912,...,0.924671,0.942481,0.969607,1.006917,1.053552,1.124739,1.195372,1.264141,1.373398,1.513360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-05 19:00:00,-0.290834,-0.107880,0.160127,0.293250,0.427659,0.556491,0.612959,0.643175,0.681565,0.710139,...,1.883750,1.956562,2.063032,2.169630,2.276425,2.380027,2.513812,2.640613,2.797703,3.112133
2018-02-05 20:00:00,-0.272896,-0.045194,0.236557,0.345973,0.458449,0.543008,0.599908,0.647607,0.674285,0.703475,...,1.748834,1.793061,1.845889,1.914256,1.998678,2.111100,2.215234,2.329767,2.528787,2.834394
2018-02-05 21:00:00,-0.194810,-0.012941,0.236399,0.362111,0.468491,0.535901,0.586688,0.630361,0.662052,0.707326,...,1.671587,1.722951,1.763096,1.810743,1.865619,1.952006,2.054039,2.169126,2.342668,2.630838


## Gaussian Mixture Fit


In [7]:
from scipy.interpolate import interp1d
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm


# Create the GMM forecasts
#gmm_timerange = ['2017-05-12', '2017-05-22']
gmm_timerange = ['2017-05-01', '2017-08-01']
forecast_gmm = forecast_smoothed.loc[gmm_timerange[0] : gmm_timerange[1]].copy()

weights_gmm = []

quantile_levels = forecast_gmm.columns.astype(float)  # Quantile levels (e.g., 1% to 99%)
for t, quantile_values in forecast_gmm.iterrows():

    # Step 1: Create an interpolator to map quantile levels to quantile values
    inv_cdf = interp1d(quantile_levels, quantile_values, kind='linear', fill_value="extrapolate")

    # Step 2: Generate synthetic samples via interpolation
    np.random.seed(42)
    synthetic_x = np.random.uniform(0.0, 1.0, 20000)
    synthetic_samples = inv_cdf(synthetic_x)  # Generate aritificial quantile value samples

    # Step 3: Fit GMM to synthetic samples
    gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42, reg_covar=1e-4)
    gmm.fit(synthetic_samples.reshape(-1, 1))

    # Extract parameters
    weights = gmm.weights_
    means = gmm.means_.flatten()
    variances = gmm.covariances_.flatten()

    # Transform variance to standard deviation
    standard_devs = np.sqrt(variances)

    weights_gmm.append([weights[0], means[0], standard_devs[0], weights[1], means[1], standard_devs[1]])


# Create a dataframe with the GMM weights
columns = ['w1', 'mu1', 'std1', 'w2', 'mu2', 'std2']

df_weights_gmm = pd.DataFrame(weights_gmm, index=forecast_gmm.index, columns=columns)


# Extract the expected values from the GMM weights and shift the GMM fits to have an expected value of 0
weights_gmm_shifted = []
expected_vals_gmm = []

for t, weights in df_weights_gmm.iterrows():
    w1, mu1, std1, w2, mu2, std2 = weights
    expected_value = w1 * mu1 + w2 * mu2
    mu1_shifted = mu1 - expected_value
    mu2_shifted = mu2 - expected_value
    weights_gmm_shifted.append([w1, mu1_shifted, std1, w2, mu2_shifted, std2])
    expected_vals_gmm.append([expected_value])

df_weights_gmm_shifted = pd.DataFrame(weights_gmm_shifted, index=df_weights_gmm.index, columns=columns)
df_exp_val_gmm = pd.DataFrame(expected_vals_gmm, index=df_weights_gmm.index, columns=['expected_value'])


In [ ]:
# save the GMM forecasts 
folder_name = f'gmm2_forecast_{current_date}_hour_1'
os.makedirs('../data/parametric_forecasts/' + folder_name, exist_ok=True)

df_exp_val_gmm.to_csv(f'../data/parametric_forecasts/{folder_name}/expected_value_forecast.csv')
df_weights_gmm_shifted.to_csv(f'../data/parametric_forecasts/{folder_name}/cdf_weights.csv')

### Automated Run


In [12]:
for i in range(1,24):
    hour = str(i)
    path_forecast = '../data/quantile_forecasts/infinite_horizon_forecast/patch_tst_2025-03-04_12-40-56_prosumption_hour_' + hour + '.csv'
    path_gt = '../data/ground_truth/residential4_prosumption.csv'


    forecast = read_in_data(path_forecast)
    forecast.columns = forecast.columns.str.replace('0_', '') # rename columns 0_0.01- 0_0.99 to 0.01 - 0.99
    forecast.columns = forecast.columns.astype(float)

    # Get the current date
    current_date = datetime.now().strftime('%Y-%m-%d')

    gt = read_in_data(path_gt)
    forecast_sorted = sort_quantiles(forecast)
    forecast_smoothed = smooth_quantiles(forecast_sorted)



    gmm_timerange = ['2017-05-01', '2017-08-01']
    forecast_gmm = forecast_smoothed.loc[gmm_timerange[0] : gmm_timerange[1]].copy()

    weights_gmm = []

    quantile_levels = forecast_gmm.columns.astype(float)  # Quantile levels (e.g., 1% to 99%)
    for t, quantile_values in forecast_gmm.iterrows():

        # Step 1: Create an interpolator to map quantile levels to quantile values
        inv_cdf = interp1d(quantile_levels, quantile_values, kind='linear', fill_value="extrapolate")

        # Step 2: Generate synthetic samples via interpolation
        np.random.seed(42)
        synthetic_x = np.random.uniform(0.0, 1.0, 20000)
        synthetic_samples = inv_cdf(synthetic_x)  # Generate aritificial quantile value samples

        # Step 3: Fit GMM to synthetic samples
        gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42, reg_covar=1e-4)
        gmm.fit(synthetic_samples.reshape(-1, 1))

        # Extract parameters
        weights = gmm.weights_
        means = gmm.means_.flatten()
        variances = gmm.covariances_.flatten()

        # Transform variance to standard deviation
        standard_devs = np.sqrt(variances)

        weights_gmm.append([weights[0], means[0], standard_devs[0], weights[1], means[1], standard_devs[1]])


    # Create a dataframe with the GMM weights
    columns = ['w1', 'mu1', 'std1', 'w2', 'mu2', 'std2']

    df_weights_gmm = pd.DataFrame(weights_gmm, index=forecast_gmm.index, columns=columns)


    # Extract the expected values from the GMM weights and shift the GMM fits to have an expected value of 0
    weights_gmm_shifted = []
    expected_vals_gmm = []

    for t, weights in df_weights_gmm.iterrows():
        w1, mu1, std1, w2, mu2, std2 = weights
        expected_value = w1 * mu1 + w2 * mu2
        mu1_shifted = mu1 - expected_value
        mu2_shifted = mu2 - expected_value
        weights_gmm_shifted.append([w1, mu1_shifted, std1, w2, mu2_shifted, std2])
        expected_vals_gmm.append([expected_value])

    df_weights_gmm_shifted = pd.DataFrame(weights_gmm_shifted, index=df_weights_gmm.index, columns=columns)
    df_exp_val_gmm = pd.DataFrame(expected_vals_gmm, index=df_weights_gmm.index, columns=['expected_value'])

    # save the GMM forecasts 
    folder_name = f'gmm2_forecast_{current_date}_hour_'+hour
    os.makedirs('../data/parametric_forecasts/' + folder_name, exist_ok=True)

    df_exp_val_gmm.to_csv(f'../data/parametric_forecasts/{folder_name}/expected_value_forecast.csv')
    df_weights_gmm_shifted.to_csv(f'../data/parametric_forecasts/{folder_name}/cdf_weights.csv')

